In [2]:
import numpy as np
import torch
import torch.nn as nn
from model import nts_RNN, nts_MLP
from utils import TimeseriesDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [3]:
def evaluate_model_on_trainset(model_name, hidden_size, seq_len, fut_len, MLP_flag):
    dataset_name='TrainSet.mat'

    if MLP_flag:
        model = nts_MLP(hidden_size, input_size=seq_len, fut_len=fut_len)
    else:
        model = nts_RNN(hidden_size, fut_len)

    model.load_state_dict(torch.load("trained_models/"+model_name))

    dataset = TimeseriesDataset(dataset_name, seq_len, fut_len)
    batch_size = 1
    loader = DataLoader(dataset, batch_size, shuffle=False)
    loss_function = nn.MSELoss()
    loss = np.zeros(len(loader))
    model.eval()

    for i, (X, y) in enumerate(loader):
        if MLP_flag:
            X = torch.swapaxes(X, 1, 2)
        output = model(X)
        if MLP_flag:
            output = torch.swapaxes(output, 1, 2)
        loss[i] = loss_function(output.detach().flatten(), y.flatten())

    return np.average(loss)

In [7]:
loss_1 = evaluate_model_on_trainset('L3_double_S10_B50_LR5e3_NR.pt', [16, 14, 12], 10, 10, MLP_flag=False)
loss_2 = evaluate_model_on_trainset('L3_four_S10_B50_LR5e3_NR.pt', [32, 28, 24], 10, 10, MLP_flag=False)
loss_3 = evaluate_model_on_trainset('MLP3_double_S10_B50_LR5e3_NR.pt', [16, 14, 12], 10, 10, MLP_flag=True)
loss_4 = evaluate_model_on_trainset('MLP3_four_S10_B50_LR2e3_NR.pt', [32, 28, 24], 10, 10, MLP_flag=True)
print([loss_1, loss_2, loss_3, loss_4])

[0.07030681471583465, 0.061816445927135234, 0.08065034566463353, 0.051099979198674296]


In [9]:
loss_1 = evaluate_model_on_trainset('L3_four_S10_B50_LR5e3_NR.pt', [32, 28, 24], 10, 10, MLP_flag=False)
loss_2 = evaluate_model_on_trainset('L3_four_S40_B50_LR5e3_NR.pt', [32, 28, 24], 40, 10, MLP_flag=False)
loss_3 = evaluate_model_on_trainset('L3_four_S80_B50_LR2e3_NR.pt', [32, 28, 24], 80, 10, MLP_flag=False)

loss_4 = evaluate_model_on_trainset('MLP3_four_S10_B50_LR2e3_NR.pt', [32, 28, 24], 10, 10, MLP_flag=True)
loss_5 = evaluate_model_on_trainset('MLP3_four_S40_B50_LR2e3_NR.pt', [32, 28, 24], 40, 10, MLP_flag=True)
loss_6 = evaluate_model_on_trainset('MLP3_four_S80_B50_LR2e3_NR.pt', [32, 28, 24], 80, 10, MLP_flag=True)
print([loss_1, loss_2, loss_3, loss_4, loss_5, loss_6])

[0.061816445927135234, 0.2326397165326853, 0.235032166603661, 0.051099979198674296, 0.04061354268580948, 0.02852294441956928]
